# Извлечение признаков из nanoGPT

In [1]:
from backbones.nanoGPT import GPT, GPTConfig

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
model = GPT.from_pretrained("gpt2")

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 123.65M


In [4]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
text = "You are a minecraft agent, you will be given an image history and actions history along with the instruction what to do. Your goal is to predict an action" * 32
token_num = len(enc.encode(text))
print(token_num)
print(token_num <= 1024)


1024
True


Как можем видеть, в гпт влезет довольно большая инструкция

Теперь имплементируем простой извлекатель признаков. Для этого скорректируем исходник nanoGPT, модифицируем метод forward:

```python
def forward(self, idx, targets=None, only_latents = False):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        
        if only_latents:
            #let's return as tuple to keep the signature of the return
            return None, None, x
        
        ...
```

In [ ]:
import torch.nn as nn
import torch

class NanoGPTFeatureExtractor(nn.Module):
    def __init__(self, device = "cuda", *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = GPT.from_pretrained("gpt2").to(device)
        self.device = device
        self.tokenizer = tiktoken.get_encoding("gpt2")
        self.cache = {}